In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [10]:
hop_teaming = pd.read_csv('DocGraph_Hop_Teaming_2018.csv', 
                          nrows = 100)
hop_teaming.head(25)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880
5,1508252933,1730166109,16,16,78.750,103.169
6,1508076001,1730166109,46,48,72.625,83.263
7,1508085911,1730166125,58,67,23.925,43.923
8,1508093394,1730166125,15,15,37.400,79.602
9,1508132986,1730166125,11,11,97.364,62.152


In [11]:
hop_teaming.dtypes

from_npi               int64
to_npi                 int64
patient_count          int64
transaction_count      int64
average_day_wait     float64
std_day_wait         float64
dtype: object

In [15]:
db = sqlite3.connect('hop_teaming.sqlite')

dtypes = {'from_npi': 'int64', 'to_npi': 'int64', 'patient_count': 'int64', 'transaction_count': 'int64', 'average_day_wait': 'float64', 'std_day_wait': 'float64'}
for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv', dtype=dtypes, chunksize=1000000)):
    chunk.to_sql('hop', db, if_exists = 'append', index = False) 


0it [00:00, ?it/s]

In [5]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [7]:
#db = sqlite3.connect('hop_teaming.sqlite')

#for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)): 
    #chunk.to_sql('hop', db, if_exists = 'append', index = False) 

0it [00:00, ?it/s]

In [ ]:
# create an empty dataframe to hold the filtered data
referrals2 = pd.DataFrame()

# read the csv file in chunks and filter the data
for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize=10000)):
    chunk = chunk.loc[(chunk['transaction_count'] >= 25) & (chunk['std_day_wait'] < 90)]
    referrals2 = pd.concat([referrals2, chunk])

# save the filtered data as a pandas dataframe
referrals2.to_csv('filtered_data.csv', index=False)
referrals2.head()

0it [00:00, ?it/s]

In [16]:
# Establish a connection to the SQLite database
db = sqlite3.connect('hop_teaming.sqlite')

# Create a cursor object to execute SQL queries
cursor = db.cursor()

# SQL query and fetch results
query = "SELECT * FROM hop WHERE transaction_count >= 50 AND average_day_wait < 50;"
results = cursor.execute(query).fetchall()

# Convert to pandas df
hop_teaming = pd.DataFrame(results, columns=[column[0] for column in cursor.description])

db.close()
hop_teaming.head(10)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508085911,1730166125,58,67,23.925,43.923
1,1508167040,1730166125,51,51,28.196,52.876
2,1508863549,1730166125,340,391,18.302,42.422
3,1508867870,1730166125,50,79,12.658,26.402
4,1508011040,1730166224,132,145,8.579,28.053
5,1497971881,1730166224,120,135,49.822,61.955
6,1508163577,1730166224,67,127,8.346,26.050
7,1508159674,1730166224,145,212,2.939,10.660
8,1508205808,1730166224,48,64,14.156,35.859
9,1508170853,1730166307,81,115,32.930,56.683


In [17]:
len(hop_teaming)

34176938

In [4]:
# This takes about 4 minutes to complete on my computer. Need to check for mistakes but might be a start.

# Monica's magic loop!
def get_primary_taxonomy(row):
    for n in range(1, 16):
        if row['primary_taxonomy'] == '':
            x = str(n)
            if row[f'Healthcare Provider Primary Taxonomy Switch_{x}'] == 'Y':
                return row[f'Healthcare Provider Taxonomy Code_{x}']
    return row['primary_taxonomy']

# Define list of relevant columns
cols = ['NPI', 'Entity Type Code', 
        'Provider Organization Name (Legal Business Name)',
        'Provider Last Name (Legal Name)', 'Provider First Name', 
        'Provider Middle Name', 'Provider Name Prefix Text',
        'Provider Name Suffix Text', 'Provider Credential Text',
        'Provider First Line Business Practice Location Address',
        'Provider Second Line Business Practice Location Address',
        'Provider Business Practice Location Address City Name',
        'Provider Business Practice Location Address State Name',
        'Provider Business Practice Location Address Postal Code']

# Append taxonomy columns 1 to 15
for i in range(1, 16):
    cols.append(f'Healthcare Provider Primary Taxonomy Switch_{i}')
    cols.append(f'Healthcare Provider Taxonomy Code_{i}')

# Read in CSV and relevant columns in chunks
chunksize = 1000000
npi_chunks = pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = cols, low_memory = False, chunksize = chunksize)

# Define empty dataframe
npi_test = pd.DataFrame()

# Loop over the chunks and process the data
for chunk in npi_chunks:
    # Process data with apply function
    chunk['primary_taxonomy'] = ''
    chunk['primary_taxonomy'] = chunk.apply(get_primary_taxonomy, axis = 1)

    # Append processed data to the empty dataframe
    npi_test = pd.concat([npi_test, chunk])

# Read in other csv files with type cleaning    
nucc = pd.read_csv('nucc_taxonomy_230.csv', encoding = 'latin1')
zip = pd.read_csv('ZIP_CBSA_122021.csv', dtype = {'zip': str})
zip['cbsa'] = zip['cbsa'].astype(str)

# Merge and edit zip codes
merged_df = pd.merge(npi_test, nucc[['Code', 'Classification']], left_on = 'primary_taxonomy', right_on = 'Code', how = 'left')
merged_df['Provider Business Practice Location Address Postal Code'] = merged_df['Provider Business Practice Location Address Postal Code'].str[:5]
merged_df = pd.merge(merged_df, zip[['zip', 'cbsa']], left_on = 'Provider Business Practice Location Address Postal Code', right_on = 'zip', how = 'left')

merged_df.tail(25)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,...,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,primary_taxonomy,Code,Classification,zip,cbsa
8420052,1598302184,1.0,NaN,LINSKER,ER,NaN,NaN,NaN,"LMSW, MFA",31 WASHINGTON SQ W,...,NaN,NaN,NaN,NaN,NaN,1041C0700X,1041C0700X,Social Worker,10011,35620
8420053,1871748715,1.0,NaN,PHILLIPS,LESLIE,ANN,DR.,NaN,O.D.,7268 JARNIGAN RD,...,NaN,NaN,NaN,NaN,NaN,152W00000X,152W00000X,Optometrist,37421,16860
8420054,1417547753,1.0,NaN,PETITT,JORDAN,NICOLE,NaN,NaN,LMFT,2904 ROWENA AVE,...,NaN,NaN,NaN,NaN,NaN,101YM0800X,101YM0800X,Counselor,90039,31080
8420055,1780017483,1.0,NaN,PREVIL,LUCKSON,NaN,DR.,NaN,O.D.,7268 JARNIGAN RD,...,NaN,NaN,NaN,NaN,NaN,152W00000X,152W00000X,Optometrist,37421,16860
8420056,1548719503,1.0,NaN,THOMPSON,PAIGE,NaN,DR.,NaN,OD,7268 JARNIGAN RD,...,NaN,NaN,NaN,NaN,NaN,152W00000X,152W00000X,Optometrist,37421,16860
8420057,1932492303,1.0,NaN,WHITMIRE,WILLIAM,CHALKLEY,DR.,NaN,M.D.,9453 DAYTON PIKE,...,NaN,NaN,NaN,NaN,NaN,207W00000X,207W00000X,Ophthalmology,37379,99999
8420058,1932492303,1.0,NaN,WHITMIRE,WILLIAM,CHALKLEY,DR.,NaN,M.D.,9453 DAYTON PIKE,...,NaN,NaN,NaN,NaN,NaN,207W00000X,207W00000X,Ophthalmology,37379,16860
8420059,1831807494,2.0,LARRUA BEHAVIOR SERVICES LLC,NaN,NaN,NaN,NaN,NaN,NaN,12484 NW SOUTH RIVER DR STE 550A,...,NaN,NaN,NaN,NaN,NaN,103K00000X,103K00000X,Behavior Analyst,33178,33100
8420060,1417007691,1.0,NaN,YANNIS,REX,A,DR.,NaN,M.D.,7268 JARNIGAN RD,...,NaN,NaN,NaN,NaN,NaN,207WX0200X,207WX0200X,Ophthalmology,37421,16860
8420061,1265102255,1.0,NaN,KAIPOV,MYRNA,JUDITH,NaN,NaN,NaN,3857 MARTIN WAY E,...,NaN,NaN,NaN,NaN,NaN,101Y00000X,101Y00000X,Counselor,98506,36500


In [8]:
# Drop Taxonomy columns
cols_to_drop = ['Healthcare Provider Primary Taxonomy Switch_{}'.format(i) for i in range(1, 16)]
cols_to_drop += ['Healthcare Provider Taxonomy Code_{}'.format(i) for i in range(1, 16)]
cols_to_drop += ['Code', 'zip']
merged_df.drop(columns=cols_to_drop, inplace=True)
merged_df.tail()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,primary_taxonomy,Code,Classification,zip,cbsa
8420048,1558068700,2.0,ELDER CARE PHYSICIANS LLC,NaN,NaN,NaN,NaN,NaN,NaN,1S072 LUTHER AVE,NaN,LOMBARD,IL,60148,207R00000X,207R00000X,Internal Medicine,60148,16980
8420049,1912011131,1.0,NaN,MCCARTY,ZACHARY,STUART,DR.,NaN,OD,7268 JARNIGAN RD,SUITE 200,CHATTANOOGA,TN,37421,152W00000X,152W00000X,Optometrist,37421,16860
8420050,1902474927,1.0,NaN,BRYANT,STEWART,NaN,DR.,NaN,MD,430 BROADWAY ST,NaN,REDWOOD CITY,CA,94063,207X00000X,207X00000X,Orthopaedic Surgery,94063,41860
8420051,1679577324,1.0,NaN,PETERSON,EDWARD,ANDREW,DR.,NaN,MD,7268 JARNIGAN RD,SUITE 200,CHATTANOOGA,TN,37421,207W00000X,207W00000X,Ophthalmology,37421,16860
8420052,1598302184,1.0,NaN,LINSKER,ER,NaN,NaN,NaN,"LMSW, MFA",31 WASHINGTON SQ W,NaN,NEW YORK,NY,10011,1041C0700X,1041C0700X,Social Worker,10011,35620
8420053,1871748715,1.0,NaN,PHILLIPS,LESLIE,ANN,DR.,NaN,O.D.,7268 JARNIGAN RD,SUITE 200,CHATTANOOGA,TN,37421,152W00000X,152W00000X,Optometrist,37421,16860
8420054,1417547753,1.0,NaN,PETITT,JORDAN,NICOLE,NaN,NaN,LMFT,2904 ROWENA AVE,NaN,LOS ANGELES,CA,90039,101YM0800X,101YM0800X,Counselor,90039,31080
8420055,1780017483,1.0,NaN,PREVIL,LUCKSON,NaN,DR.,NaN,O.D.,7268 JARNIGAN RD,SUITE 200,CHATTANOOGA,TN,37421,152W00000X,152W00000X,Optometrist,37421,16860
8420056,1548719503,1.0,NaN,THOMPSON,PAIGE,NaN,DR.,NaN,OD,7268 JARNIGAN RD,SUITE 200,CHATTANOOGA,TN,37421,152W00000X,152W00000X,Optometrist,37421,16860
8420057,1932492303,1.0,NaN,WHITMIRE,WILLIAM,CHALKLEY,DR.,NaN,M.D.,9453 DAYTON PIKE,NaN,SODDY DAISY,TN,37379,207W00000X,207W00000X,Ophthalmology,37379,99999


In [107]:
#merged_df['Entity Type Code'] = merged_df['Entity Type Code'].astype(str).str.replace('.0', '')
#merged_df.tail(25)

/var/folders/y2/1hbr5nhn5g74yh6sf24m5s9xr861g0/T/ipykernel_6813/1902387560.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_df['Entity Type Code'] = merged_df['Entity Type Code'].astype(str).str.replace('.0', '')


,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,primary_taxonomy,Code,Classification,zip,cbsa
8420052,1598302184,1,NaN,LINSKER,ER,NaN,NaN,NaN,"LMSW, MFA",31 WASHINGTON SQ W,NaN,NEW YORK,NY,10011,104100000X,N,1041C0700X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1041C0700X,1041C0700X,Social Worker,10011,35620
8420053,1871748715,1,NaN,PHILLIPS,LESLIE,ANN,DR.,NaN,O.D.,7268 JARNIGAN RD,SUITE 200,CHATTANOOGA,TN,37421,152W00000X,N,152W00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152W00000X,152W00000X,Optometrist,37421,16860
8420054,1417547753,1,NaN,PETITT,JORDAN,NICOLE,NaN,NaN,LMFT,2904 ROWENA AVE,NaN,LOS ANGELES,CA,90039,101YM0800X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101YM0800X,101YM0800X,Counselor,90039,31080
8420055,1780017483,1,NaN,PREVIL,LUCKSON,NaN,DR.,NaN,O.D.,7268 JARNIGAN RD,SUITE 200,CHATTANOOGA,TN,37421,152W00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152W00000X,152W00000X,Optometrist,37421,16860
8420056,1548719503,1,NaN,THOMPSON,PAIGE,NaN,DR.,NaN,OD,7268 JARNIGAN RD,SUITE 200,CHATTANOOGA,TN,37421,152W00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152W00000X,152W00000X,Optometrist,37421,16860
8420057,1932492303,1,NaN,WHITMIRE,WILLIAM,CHALKLEY,DR.,NaN,M.D.,9453 DAYTON PIKE,NaN,SODDY DAISY,TN,37379,207W00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207W00000X,207W00000X,Ophthalmology,37379,99999
8420058,1932492303,1,NaN,WHITMIRE,WILLIAM,CHALKLEY,DR.,NaN,M.D.,9453 DAYTON PIKE,NaN,SODDY DAISY,TN,37379,207W00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207W00000X,207W00000X,Ophthalmology,37379,16860
8420059,1831807494,2,LARRUA BEHAVIOR SERVICES LLC,NaN,NaN,NaN,NaN,NaN,NaN,12484 NW SOUTH RIVER DR STE 550A,NaN,MEDLEY,FL,33178,103K00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103K00000X,103K00000X,Beh

In [23]:
# Filter Nashville Hospitals
filtered_df = merged_df[(merged_df['Provider Business Practice Location Address City Name'] == 'NASHVILLE') & 
                        (merged_df['Provider Business Practice Location Address State Name'] == 'TN') &
                        (merged_df['Entity Type Code'] == 2.0)]
len(filtered_df)

3017

In [25]:
# Merge referrals
referrals = pd.merge(hop_teaming, filtered_df, how='inner', left_on='to_npi', right_on='NPI')
referrals.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,primary_taxonomy,Code,Classification,zip,cbsa
0,1508821026,1770667289,62,84,18.143,36.871,1770667289,2.0,MIDDLE TENNESSEE IMAGING LLC,NaN,NaN,NaN,NaN,NaN,NaN,28 WHITE BRIDGE ROAD,SUITE 111,NASHVILLE,TN,37205,2085R0202X,2085R0202X,Radiology,37205,34980
1,1508804337,1770667289,1772,1881,6.081,28.247,1770667289,2.0,MIDDLE TENNESSEE IMAGING LLC,NaN,NaN,NaN,NaN,NaN,NaN,28 WHITE BRIDGE ROAD,SUITE 111,NASHVILLE,TN,37205,2085R0202X,2085R0202X,Radiology,37205,34980
2,1508890286,1770667289,162,255,33.737,45.758,1770667289,2.0,MIDDLE TENNESSEE IMAGING LLC,NaN,NaN,NaN,NaN,NaN,NaN,28 WHITE BRIDGE ROAD,SUITE 111,NASHVILLE,TN,37205,2085R0202X,2085R0202X,Radiology,37205,34980
3,1508884875,1770667289,135,164,35.159,50.708,1770667289,2.0,MIDDLE TENNESSEE IMAGING LLC,NaN,NaN,NaN,NaN,NaN,NaN,28 WHITE BRIDGE ROAD,SUITE 111,NASHVILLE,TN,37205,2085R0202X,2085R0202X,Radiology,37205,34980
4,1508246364,1770667289,44,63,22.619,37.623,1770667289,2.0,MIDDLE TENNESSEE IMAGING LLC,NaN,NaN,NaN,NaN,NaN,NaN,28 WHITE BRIDGE ROAD,SUITE 111,NASHVILLE,TN,37205,2085R0202X,2085R0202X,Radiology,37205,34980


In [26]:
len(referrals)

72275

In [31]:
# Filter Entity Code 1.0
referred_to = merged_df[(merged_df['Entity Type Code'] == 1.0)]
len(referred_to)

6370379

In [33]:
# Merge to combine all Nashville, TN referrals
referrals_merged = referrals.merge(referred_to, how='left', left_on='from_npi', right_on='NPI', suffixes=('_referred_to', '_referred_by'))
len(referrals_merged)

78817

In [ ]:
referrals_merged.head(10)

In [45]:
referrals_merged['Provider Organization Name (Legal Business Name)_referred_to'].value_counts().head(20)

VANDERBILT UNIVERSITY MEDICAL CENTER      9954
ASSOCIATED PATHOLOGISTS, LLC              6279
PATHGROUP LABS, LLC                       4825
TENNESSEE ONCOLOGY PLLC                   4753
SAINT THOMAS MEDICAL PARTNERS             4225
ASSOCIATED PATHOLOGISTS LLC               4135
AEGIS SCIENCES CORPORATION                3223
RADIOLOGY ALLIANCE PC                     3096
WELLNESS SOLUTIONS GERIATRICS, PLLC       2099
NEPHROLOGY ASSOCIATES, PC                 2066
ANESTHESIA MEDICAL GROUP, PC              2047
CENTENNIAL HEART LLC                      1891
SAINT THOMAS WEST HOSPITAL                1864
HCA HEALTH SERVICES OF TENNESSEE, INC.    1789
DCI LABORATORY LLC                        1462
TENNESSEE ORTHOPAEDIC ALLIANCE PA         1181
HERITAGE MEDICAL ASSOCIATES PC            1172
ACCURATE HEALTHCARE, INC.                 1065
UROLOGY ASSOCIATES PC                      800
THE SURGICAL CLINIC PLLC                   755
Name: Provider Organization Name (Legal Business Name)_refer

In [37]:
grouped = referrals_merged.groupby(['Classification_referred_by', 'Provider Organization Name (Legal Business Name)_referred_to']).size()
sorted_counts = grouped.sort_values(ascending=False)
sorted_counts

Classification_referred_by  Provider Organization Name (Legal Business Name)_referred_to
Internal Medicine           VANDERBILT UNIVERSITY MEDICAL CENTER                            2234
                            TENNESSEE ONCOLOGY PLLC                                         1243
                            ASSOCIATED PATHOLOGISTS, LLC                                    1164
Nurse Practitioner          VANDERBILT UNIVERSITY MEDICAL CENTER                            1017
Internal Medicine           SAINT THOMAS MEDICAL PARTNERS                                    874
                                                                                            ... 
Podiatrist                  MIDDLE TENNESSEE IMAGING, LLC                                      1
                            MIDDLE TENNESSEE INTERNAL MEDICINE ASSOCIATES, PLLC                1
                            MONADNOCK EMERGENCY PHYSICIANS PLLC                                1
                            NASHVILLE 

In [38]:
sorted_counts.head(20)

Classification_referred_by               Provider Organization Name (Legal Business Name)_referred_to
Internal Medicine                        VANDERBILT UNIVERSITY MEDICAL CENTER                            2234
                                         TENNESSEE ONCOLOGY PLLC                                         1243
                                         ASSOCIATED PATHOLOGISTS, LLC                                    1164
Nurse Practitioner                       VANDERBILT UNIVERSITY MEDICAL CENTER                            1017
Internal Medicine                        SAINT THOMAS MEDICAL PARTNERS                                    874
                                         PATHGROUP LABS, LLC                                              808
Family Medicine                          VANDERBILT UNIVERSITY MEDICAL CENTER                             778
Internal Medicine                        ASSOCIATED PATHOLOGISTS LLC                                      716
Nurse Practitioner

In [39]:
referrals_merged['Provider Business Practice Location Address State Name_referred_by'].value_counts()

TN    38406
KY     2286
GA     1774
AL      692
MS      651
NC      598
MO      555
IN      515
TX      509
FL      468
OH      410
VA      387
SC      367
CA      213
AR      213
LA      199
IL      194
NY      176
PA      171
MN      169
WV      167
MI      159
MA      123
CO      105
MD      102
AZ       76
WA       73
CT       68
NJ       62
WI       61
KS       57
NE       34
NV       33
UT       33
NM       32
OR       28
MT       27
DE       23
ME       22
OK       20
IA       15
NH       15
DC       14
HI       12
AK        7
ID        5
RI        5
GU        2
SD        2
Name: Provider Business Practice Location Address State Name_referred_by, dtype: int64

In [40]:
referrals_merged['Provider Business Practice Location Address State Name_referred_to'].value_counts()

TN    78817
Name: Provider Business Practice Location Address State Name_referred_to, dtype: int64

In [56]:
# Filter out Vanderbilt 
sorted_counts_NV = grouped_NV.reset_index().sort_values(by=['Provider Organization Name (Legal Business Name)_referred_to', 'Classification_referred_by'], ascending=[False, True], inplace=False)
sorted_counts_NV = sorted_counts_NV.loc[sorted_counts_NV['Provider Organization Name (Legal Business Name)_referred_to'] != 'VANDERBILT UNIVERSITY MEDICAL CENTER']
sorted_counts_NV = sorted_counts_NV.sort_values(by=0, ascending=False)
sorted_counts_NV.head(20)


,Classification_referred_by,Provider Organization Name (Legal Business Name)_referred_to,0
685,Internal Medicine,TENNESSEE ONCOLOGY PLLC,1243
520,Internal Medicine,"ASSOCIATED PATHOLOGISTS, LLC",1164
667,Internal Medicine,SAINT THOMAS MEDICAL PARTNERS,874
650,Internal Medicine,"PATHGROUP LABS, LLC",808
519,Internal Medicine,ASSOCIATED PATHOLOGISTS LLC,716
953,Nurse Practitioner,"PATHGROUP LABS, LLC",643
639,Internal Medicine,"NEPHROLOGY ASSOCIATES, PC",617
659,Internal Medicine,RADIOLOGY ALLIANCE PC,610
362,Family Medicine,"PATHGROUP LABS, LLC",582
290,Family Medicine,"ASSOCIATED PATHOLOGISTS, LLC",564


In [57]:
len(sorted_counts_NV)

2121

In [68]:
# wrong zip codes
mask = merged_df['Provider Business Practice Location Address Postal Code'].astype(str).str.len() != 5
rows_with_wrong_zip = merged_df[mask]
rows_with_wrong_zip

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,primary_taxonomy,Code,Classification,zip,cbsa
13013,1356341580,1.0,NaN,VERAY,FRANCISCO,XAVIER,DR.,IV,"D.D.S, M.S",PSC 827 BOX526,NaN,FPO,GRICIGNANO,AE,,NaN,NaN,NaN,NaN
26932,1700875341,1.0,NaN,SIM,HYUN,S,NaN,NaN,M.D.,"121 GENERAL HOSPITAL, BOX 556",NaN,SEOUL,SOUTH KOREA,140,207L00000X,207L00000X,Anesthesiology,NaN,NaN
35244,1629059217,1.0,NaN,RIMEHAUG,STEIN,A,MR.,NaN,PT,POSTBOKS 1405,FYSIOTERAPITJENESTEN,FREDRIKSTAD,OSTFOLD,1605,225100000X,225100000X,Physical Therapist,NaN,NaN
38147,1881678126,1.0,NaN,VILLAFRANCA,VERONICA,NaN,MS.,NaN,"MSN, RN",HHC 18TH MEDCOM,ATTN: DCCS-QM (CREDENTIALS),APO,PACIFIC,AP,163WC0200X,163WC0200X,Registered Nurse,NaN,NaN
41158,1295719052,1.0,NaN,SEIGNE,PATRICK,WILLIAM,DR.,NaN,M.D.,"DEPT OF ANAESTHESIA, CORK UNIVERSITY HOSPITAL,",WILTON,CORK,COUNTY CORK,0000,207LC0200X,207LC0200X,Anesthesiology,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7786701,1568843084,1.0,NaN,WOODWARD,KRISTEN,NaN,NaN,NaN,M.D.,13 RICHMOND ROAD,C/O EDGEWOOD PEDIATRICS,HAMILTON,HAMILTON,HM08,208000000X,208000000X,Pediatrics,NaN,NaN
7829527,1003306317,1.0,NaN,TSAI,NINA,NaN,DR.,NaN,DMD,VIA STAZIONE 1,NaN,BALERNA,TI,6828,1223P0300X,1223P0300X,Dentist,NaN,NaN
7830532,1316601610,1.0,NaN,GEORGE,LISSY,NaN,NaN,NaN,NaN,9/3 KNOX STREET,NaN,HAMILTON CENTRAL,WAIKATO,3204,103TC0700X,103TC0700X,Psychologist,NaN,NaN
7912821,1659566933,2.0,U.S. NAVY,NaN,NaN,NaN,NaN,NaN,NaN,"3RD MED BN, 3RD MLG","HANSEN CLINIC, UNIT 38448",OKINAWA,FPO,AP,261QM1102X,261QM1102X,Clinic/Center,NaN,NaN


In [69]:
merged_df['zip'].isna().sum()

14473